### 1.2 UM-Bridge Model Integration

In this section, UM-Bridge model server can be found in the following. It is used to solve the steady state temperature distribution by considering the value of $k$ and number of grid of points as inputs. 

In [ ]:
# umbridge server
import nest_asyncio
nest_asyncio.apply()
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import scipy.special as special
from mpl_toolkits.mplot3d import axes3d
from numpy import exp,arange
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show
from scipy.linalg import block_diag
import numpy
import umbridge
from scipy import linalg

class PDEModel(umbridge.Model):
    def __init__(self):
        super().__init__("forward")
        self.L = 1.2
        self.H = 1.2
        self.g = 4 * 10**6
        self.T_0 = 200
        self.h_val = 0.1 
        self.N = 12
        
        
    def get_input_sizes(self, config):
        return [1]  # for the values k 
        
    def get_output_sizes(self, config):
        return [9]  # for the output variables T
        
    def __call__(self, parameters, config):
        k = parameters[0][0]
        A = self.f(self.N, config)
        b = self.h(self.N, self.g, k, config)
        T = self.solve(A, b, config)
        d1=np.reshape(T, (self.N - 1, self.N -1))
        d2 = np.zeros((self.N + 1, self.N + 1))
        d2[0] = self.T_0
        d2[1:-1, 1:-1]=d1
        self.Y = []
        for i in range(3,12,3):
            for j in range(3,12,3):
                d2[i,j]
                self.Y.append(d2[i,j])
                j = j+3
            i=i+3
            
        self.Y = [float(x) for x in self.Y]
        return [self.Y]
        
    # coefficient matrix
    def f(self, n, config):
        Ddiag  = -4 * np.eye(n-1)
        Dupper = np.diag([1] * (n - 2), 1)
        Dlower = np.diag([1] * (n - 2), -1)
        D = Ddiag + Dupper + Dlower
        Ds = [D] * (n- 1)
        B1  = block_diag(*Ds)
        I = np.ones((n - 1) * (n - 2))
        Iupper = np.diag(I, n - 1)
        Ilower = np.diag(I, -n + 1)
        B2 = Iupper + Ilower
        matrix = B1 + B2
        return matrix

    
    # matrix for the right hand side of the equation
    def h(self, n, g, k, config):
        b = np.zeros((n - 1)**2)
        b[:(n-1)] = -(g/k) * (self.h_val **2) - self.T_0
        b[(n-1):(n - 1)**2]=  -(g/k) * (self.h_val **2)
        return b

    
    # solve the linear system 
    def solve(self, x, y, config):
        solution = linalg.solve(x, y)
        return solution

        
    def supports_evaluate(self):
        return True

pde_model = PDEModel()
umbridge.serve_models([pde_model], 4242) # start model server 

`======== Running on http://0.0.0.0:4242 ========
(Press CTRL+C to quit)`